<a href="https://colab.research.google.com/github/fairuznawar122/Network-Intrusion-Detection-System/blob/main/NIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Specify the path to your file
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/LDAP.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/MSSQL.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/NetBIOS.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/Portmap.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/Syn.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/UDP.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/UDPLag.csv'

# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_path, low_memory=False)

# Display the first few rows of the DataFrame
# print(df.head())


In [ ]:
df.columns = df.columns.str.strip()
print(df.columns)

Index(['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Le

In [ ]:
# Now you can access the 'Label' column
label_counts = df['Label'].value_counts()

# Display the counts
print(label_counts)


Label
LDAP       1905191
NetBIOS     202919
BENIGN        5124
Name: count, dtype: int64


# Feature Names Used

In [ ]:
feature_names = ['Fwd Packet Length Max',
                 'Fwd Packet Length Min',
                 'Min Packet Length',
                 'Max Packet Length',
                 'Average Packet Size',
                 'Fwd Packets/s',
                 'Fwd Header Length',
                 'Fwd Header Length.1',
                 'min_seg_size_forward',
                 'Total Length of Fwd Packets',
                 'Fwd Packet Length Std',
                 'Flow IAT Min',
                 'Subflow Fwd Bytes',
                 'Destination Port',
                 'Protocol',
                 'Packet Length Std',
                 'Flow Duration',
                 'Fwd IAT Total',
                 'ACK Flag Count',
                 'Init_Win_bytes_forward',
                 'Flow IAT Mean',
                 'Flow IAT Max',
                 'Fwd IAT Mean',
                 'Fwd IAT Max',
                 'Label']
len(feature_names)

25

In [ ]:
df= df[feature_names]
df.shape

(2113234, 25)

# Resampling Data to Balance it

In [ ]:
import pandas as pd
from sklearn.utils import resample

# Creating the initial dataset


# Determine the target number of samples (the number of 'BENIGN' samples)
target_size = df[df['Label'] == 'BENIGN'].shape[0]

# Resample the dataset
ldap_resampled = resample(df[df['Label'] == 'NetBIOS'],
                          replace=True,     # Sample with replacement
                          n_samples=target_size, # Match the 'BENIGN' count
                          random_state=42)    # For reproducibility

netbios_resampled = resample(df[df['Label'] == 'LDAP'],
                             replace=True,
                             n_samples=target_size,
                             random_state=42)
# syn = resample(df[df['Label'] == 'Syn'],
#                              replace=True,
#                              n_samples=target_size,
#                              random_state=42)

benign_resampled = df[df['Label'] == 'BENIGN']  # BENIGN already matches the target size

# Combine all resampled categories back into one DataFrame
resampled_df = pd.concat([netbios_resampled,ldap_resampled, benign_resampled])

# Shuffle the DataFrame to mix the samples
resampled_df = resampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new distribution
print(resampled_df['Label'].value_counts())


Label
NetBIOS    5124
BENIGN     5124
LDAP       5124
Name: count, dtype: int64


In [ ]:
resampled_df = resampled_df.reset_index(drop=True)
resampled_df

,Fwd Packet Length Max,Fwd Packet Length Min,Min Packet Length,Max Packet Length,Average Packet Size,Fwd Packets/s,Fwd Header Length,Fwd Header Length.1,min_seg_size_forward,Total Length of Fwd Packets,...,Packet Length Std,Flow Duration,Fwd IAT Total,ACK Flag Count,Init_Win_bytes_forward,Flow IAT Mean,Flow IAT Max,Fwd IAT Mean,Fwd IAT Max,Label
0,229.0,229.0,229.0,229.0,343.5,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
1,229.0,229.0,229.0,229.0,343.5,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
2,0.0,0.0,0.0,0.0,0.0,3.772105e+00,128,128,32,0.0,...,0.000000,1060416,1023970.0,0,296,212083.2,1023962.0,341323.333333,1023962.0,BENIGN
3,229.0,229.0,229.0,229.0,343.5,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
4,1472.0,1472.0,1472.0,1472.0,2208.0,2.000000e+06,28,28,14,2944.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,LDAP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15367,229.0,229.0,229.0,229.0,343.5,2.000000e+06,64,64,32,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
15368,53.0,6.0,6.0,53.0,56.0,2.000000e+06,40,40,20,59.0,...,27.135463,1,1.0,0,37800,1.0,1.0,1.000000,1.0,BENIGN
15369,229.0,229.0,229.0,229.0,343.5,2.000000e+06,40,40,20,458.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,NetBIOS
15370,1472.0,1472.0,1472.0,1472.0,2208.0,2.000000e+06,40,40,20,2944.0,...,0.000000,1,1.0,0,-1,1.0,1.0,1.000000,1.0,LDAP


In [ ]:
# Replace 'NetBIOS' with 1 and 'BENIGN' with 0 in the 'Category' column
resampled_df['Label'] = resampled_df['Label'].replace({'UDP': 1,'LDAP': 1,'NetBIOS':1, 'BENIGN': 0})
resampled_df = resampled_df.dropna()

# Display the updated DataFrame
print(resampled_df['Label'].value_counts())
resampled_df.shape

Label
1    10248
0     5124
Name: count, dtype: int64


<ipython-input-45-e0c39f749b70>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  resampled_df['Label'] = resampled_df['Label'].replace({'UDP': 1,'LDAP': 1,'NetBIOS':1, 'BENIGN': 0})


(15372, 25)

# Creating the Main Dataset

In [ ]:
resampled_df.to_csv('d1.csv', index=False)

In [ ]:
df1 = pd.read_csv('/content/m3.csv', low_memory=False)
df1.shape

(109364, 25)

In [ ]:
df_merged = pd.concat([resampled_df, df1], ignore_index=True)
df_merged.shape

(124736, 25)

# Importing the Main Dataset

In [ ]:
df = pd.read_csv('/content/ProcessedDatset.csv', low_memory=False)
print(df['Label'].value_counts())

df

Label
1    70565
0    54171
Name: count, dtype: int64


,Fwd Packet Length Max,Fwd Packet Length Min,Min Packet Length,Max Packet Length,Average Packet Size,Fwd Packets/s,Fwd Header Length,Fwd Header Length.1,min_seg_size_forward,Total Length of Fwd Packets,...,Packet Length Std,Flow Duration,Fwd IAT Total,ACK Flag Count,Init_Win_bytes_forward,Flow IAT Mean,Flow IAT Max,Fwd IAT Mean,Fwd IAT Max,Label
0,229.0,229.0,229.0,229.0,343.500000,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.000000e+00,1.0,1.000000e+00,1.0,1
1,229.0,229.0,229.0,229.0,343.500000,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.000000e+00,1.0,1.000000e+00,1.0,1
2,0.0,0.0,0.0,0.0,0.000000,3.772105e+00,128,128,32,0.0,...,0.000000,1060416,1023970.0,0,296,2.120832e+05,1023962.0,3.413233e+05,1023962.0,0
3,229.0,229.0,229.0,229.0,343.500000,2.000000e+06,16,16,8,458.0,...,0.000000,1,1.0,0,-1,1.000000e+00,1.0,1.000000e+00,1.0,1
4,1472.0,1472.0,1472.0,1472.0,2208.000000,2.000000e+06,28,28,14,2944.0,...,0.000000,1,1.0,0,-1,1.000000e+00,1.0,1.000000e+00,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124731,31.0,0.0,0.0,31.0,31.000000,2.000000e+06,64,64,32,31.0,...,17.897858,1,1.0,0,355,1.000000e+00,1.0,1.000000e+00,1.0,0
124732,6.0,6.0,6.0,6.0,7.500000,2.000000e+04,40,40,20,12.0,...,0.000000,100,49.0,1,5840,3.333333e+01,51.0,4.900000e+01,49.0,1
124733,6.0,6.0,6.0,6.0,8.000000,4.545455e+03,20,20,20,6.0,...,0.000000,220,0.0,0,296,1.100000e+02,218.0,0.000000e+00,0.0,0
124734,6.0,6.0,6.0,6.0,9.000000,1.000000e+06,40,40,20,12.0,...,0.000000,2,2.0,1,5840,2.000000e+00,2.0,2.000000e+00,2.0,1


In [ ]:
# 1. Quick list of each column’s dtype:
print(df.dtypes)

# 2. More detailed summary (including non-null counts):
df.info()


Fwd Packet Length Max          float64
Fwd Packet Length Min          float64
Min Packet Length              float64
Max Packet Length              float64
Average Packet Size            float64
Fwd Packets/s                  float64
Fwd Header Length              float64
Fwd Header Length.1            float64
min_seg_size_forward           float64
Total Length of Fwd Packets    float64
Fwd Packet Length Std          float64
Flow IAT Min                   float64
Subflow Fwd Bytes              float64
Destination Port               float64
Protocol                       float64
Packet Length Std              float64
Flow Duration                  float64
Fwd IAT Total                  float64
ACK Flag Count                 float64
Init_Win_bytes_forward         float64
Flow IAT Mean                  float64
Flow IAT Max                   float64
Fwd IAT Mean                   float64
Fwd IAT Max                    float64
Label                           object
dtype: object
<class 'pan

In [ ]:
X = df.drop(columns=["Label"])
y = df["Label"]

# 2. Force everything to numeric, turning bad values into NaN
X = X.apply(pd.to_numeric, errors="coerce")

# 3. Drop any rows that now have NaNs (either in X or y)
data = pd.concat([X, y], axis=1).dropna()
X = data[X.columns]
y = data["Label"]

# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
# 1. Find overlapping indices
dup_idx = X_train.index.intersection(X_test.index)
print(f"Removing {len(dup_idx)} duplicate rows from test set")

# 2. Drop them from both X_test and y_test
X_test_clean = X_test.drop(index=dup_idx)
y_test_clean = y_test.drop(index=dup_idx)

# 3. (Optional) Check
print("New test size:", X_test_clean.shape[0])


Removing 0 duplicate rows from test set
New test size: 24948


# Random Forest Pipeline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# 1. Split features & target
X = df.drop(columns=["Label"])
y = df["Label"]

# 2. Stratified train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 3. Build the pipeline: normalize → random forest
pipeline = Pipeline([
    ("scaler", MinMaxScaler(feature_range=(0,1))),
    ("rf", RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    )),
])

# 4. Train
pipeline.fit(X_train, y_train)

# 5. Predict & evaluate
y_pred = pipeline.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 6. (Optional) Feature importances
importances = pd.Series(
    pipeline.named_steps["rf"].feature_importances_,
    index=X.columns
).sort_values(ascending=False)
print("\nTop 10 Features:\n", importances.head(10))


Confusion Matrix:
 [[10832     3]
 [    4 14109]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     10835
           1       1.00      1.00      1.00     14113

    accuracy                           1.00     24948
   macro avg       1.00      1.00      1.00     24948
weighted avg       1.00      1.00      1.00     24948


Top 10 Features:
 Destination Port               0.237561
Flow IAT Mean                  0.085748
Init_Win_bytes_forward         0.073860
Flow IAT Max                   0.073564
Packet Length Std              0.071376
Min Packet Length              0.058738
Fwd Packet Length Min          0.052126
Total Length of Fwd Packets    0.051001
Fwd Packets/s                  0.042292
Subflow Fwd Bytes              0.042066
dtype: float64


In [ ]:
# Saving the Pipeline
joblib.dump(pipeline, 'rf_pipeline.pkl')


['rf_pipeline.pkl']